# GCAM Annual Meeting 2023 Tutorial

The purpose of this tutorial is to demonstrate how `stitches` can be used as an emulator. While `stitches` can
 emulate a number of CMIP6 models this example will focus on emulating CanESM5 SSP245 results.

To use `stitches`, there are a number of decisions users have to make,
perhaps the most important being:

* Which ESM will `stitches` emulate?
* What archive data will be used? These are values that the target data will be matched to. It should only
contain data for the specific ESM that is being emulated. Users may limit the number of experiments or
ensemble realizations within the archive in order to achieve their specific experimental setup.
* What target data will be used? This data frame represents the temperature pathway the stitched product
will follow. The contents of this data frame may come from CMIP6 ESM results for an SSP or it may follow
some arbitrary pathway.

A diagram illustrating the `stitches` process is included for reference:

![stitches workflow](figs/stitches_diagram.jpg)

- `stitches` defaults to $X=9$ year windows.


## Getting Started


In [ ]:
import stitches


Load the additional python libraries that will be used in this example.  These packages are installed as `stitches` dependencies.

In [ ]:
import os
import pkg_resources
import warnings

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt



# For help with plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6


## Install the package data from Zenodo

The package data is all data that has been processed from raw Pangeo data
 and is generated with package functions. For convenience and rapid cloning
 of the github repository, the package data is also minted on Zenodo and
 can be quickly downloaded for using the package.
 

In [ ]:
# stitches.install_package_data()

# Example Set Up

In this example, we will use `stitches` to emulate CanESM5 SSP245 results.
Then we will compare the `stitches` results with actual CMIP6 CanESM5 SSP245
output data. This is our _Target Data_.

For CMIP6 results, Earth system model data runs from 1850-2100 (or 2099,
depending on the ESM). This tutorial will focus on emulating that entire period.

#### Decide on the target data
- The primary input to `stitches` functions that most users will adjust is the
target data.

- The target data is the temperature pathway the stitched (emulated) product
will follow. This data can come from an ESM or another class of climate models,
for a specific SSP scenario or an arbitrarily defined scenario. Similarly to the archive
data, the target data should contain the mean temperature anomaly and rate of
temperature change over a window of time.

- The target data window and the archive window must be the same length,
`stitches` uses a 9-year window by default.  `stitches` includes functions for
processing raw ESM  Tgav data into the structure it needs for matching.
- In this example, we will use CanESM5 SSP245 results for a single ensemble
member to use as our target data.

In [1]:
# Load time series and subset to target time series if needed:
targ = pd.read_csv(os.path.join(data_directory, "tas-data", "CanESM5_tas.csv"))
target_data = targ.loc[(targ["model"] == "CanESM5")
                       & (targ["experiment"] == 'ssp245')].copy()

target_data  = target_data[target_data["ensemble"].isin(['r1i1p1f1'])].copy()

target_data = target_data.drop(columns='zstore').reset_index(drop=True)


NameError: name 'pd' is not defined

Take a look at the structure and a plot of the time series we will be targeting:

In [ ]:
print(target_data.head())
target_data.plot(x='year', y='global average temperature anomaly from 1995-2014 average')
plt.show()
plt.close()


- Any time series of global average temperature anomalies can be used as a
target. However, the data frame containing this time series must be structured as
above: a `variable` column containing entries of 'tas',  `year` and `value`
columns containing the data, and `experiment`, `ensemble`, `model` columns
with identifying information of the source of this target data.
- The actual entries in the `experiment`, `ensemble`, `model` columns are only
used for generating identifying strings for generated ensemble members.

- Finally, the


- In this demonstration, we will specifically be targeting ensemble member 1
of the CanESM5 SSP245 simulations. The entire SSP245 ensemble may be
jointly targeted by omitting the line
`target_data  = target_data[target_data["ensemble"].isin(['r1i1p1f1'])].copy()`

In [ ]:
#### Decide on the archive data

- Limit the archive matching data to the model we are trying to emulate, CanESM5
in this case.
- In this example, we treat SSP245 as a novel scenario rather than one
run by the ESM and available, so we exclude it from the archive data.
- The internal package data called `matching_archive` contains the temperature
results for all the ESMs-Scenarios-ensemble members that are available for
`stitches` to use in its matching process. In this file monthly, tas output has
been processed to mean temperature anomaly and the temperature change over a
window of time. By default `stitches` uses a 9-year window.

- `stitches` actually provides two files in its pacakge data.
- `matching_archive.csv` can be considered the default (for now). It takes

In [ ]:
# read in the package data of all ESMs-Scenarios-ensemble members avail.
data_directory = pkg_resources.resource_filename('stitches', "data")
path = os.path.join(data_directory, 'matching_archive_staggered.csv')
data = pd.read_csv(path)


Let's start from the final ESM year, 2100 for CanESM5, and prioritize getting that
as a full 9 year window. Short windows can be on the front end.

- if the final year for an ESM is 2099, you'd just do
`end_yr_vector = end_yr_vector-1` to shift everything up a year.

In [30]:
staggered_archive = data.copy()

end_yr_vector = [1857, 1866, 1875,1884, 1893,
                 1902, 1911, 1920, 1929, 1938, 1947,
                 1956,  1965, 1974, 1983, 1992, 2001,
                 2010, 2019, 2028, 2037, 2046, 2055,
                 2064, 2073, 2082, 2091, 2100]

tmp = staggered_archive.loc[(data["experiment"].isin(['ssp126', 'ssp370', 'ssp585']))
                       & (data["model"] == "CanESM5")].copy()
archive_data = stitches.fx_processing.subset_archive(staggered_archive = tmp,
                                            end_yr_vector = end_yr_vector)

print(archive_data)

     experiment variable    model   ensemble  start_yr  end_yr  year  \
0        ssp126      tas  CanESM5  r10i1p1f1      1858    1866  1862   
1        ssp126      tas  CanESM5  r10i1p1f1      1867    1875  1871   
2        ssp126      tas  CanESM5  r10i1p1f1      1876    1884  1880   
3        ssp126      tas  CanESM5  r10i1p1f1      1885    1893  1889   
4        ssp126      tas  CanESM5  r10i1p1f1      1894    1902  1898   
...         ...      ...      ...        ...       ...     ...   ...   
2020     ssp585      tas  CanESM5   r9i1p1f1      2056    2064  2060   
2021     ssp585      tas  CanESM5   r9i1p1f1      2065    2073  2069   
2022     ssp585      tas  CanESM5   r9i1p1f1      2074    2082  2078   
2023     ssp585      tas  CanESM5   r9i1p1f1      2083    2091  2087   
2024     ssp585      tas  CanESM5   r9i1p1f1      2092    2100  2096   

            fx        dx  
0    -1.299509 -0.001848  
1    -1.225040  0.010931  
2    -1.205325 -0.012385  
3    -1.299790  0.004692  


The staggered archive ONLY has 9 year windows, no short windows. So no window
from 1850-1857 exists to get pulled for that `end_yr`. That's fine because  the
historical period is so consistent, there are plenty of other full sized (9year)
windows that have similar properties and can make for a good match, so we aren't
really losing anything by not having archive points representing 1850-1857.
If we really want, we can go ahead and have an 1850-1858 window in the archive
by just replacing 1857 with 1858 in the above. There will be on year (1858) in
common between that new window and the one ending in 1866 but that's not
an amount of similarity that would lead to unrealistic behavior like if we used
all of the entries in the staggered archive.

And that would look like:

In [41]:
end_yr2 = [1858, 1866, 1875,1884, 1893,
           1902, 1911, 1920, 1929, 1938, 1947,
           1956,  1965, 1974, 1983, 1992, 2001,
           2010, 2019, 2028, 2037, 2046, 2055,
           2064, 2073, 2082, 2091, 2100]
x = stitches.fx_processing.subset_archive(staggered_archive = tmp,
                                      end_yr_vector = end_yr2).sort_values(by=['experiment', 'ensemble', 'end_yr'])
print(x)

     experiment variable    model   ensemble  start_yr  end_yr  year  \
0        ssp126      tas  CanESM5  r10i1p1f1      1850    1858  1854   
75       ssp126      tas  CanESM5  r10i1p1f1      1858    1866  1862   
76       ssp126      tas  CanESM5  r10i1p1f1      1867    1875  1871   
77       ssp126      tas  CanESM5  r10i1p1f1      1876    1884  1880   
78       ssp126      tas  CanESM5  r10i1p1f1      1885    1893  1889   
...         ...      ...      ...        ...       ...     ...   ...   
2095     ssp585      tas  CanESM5   r9i1p1f1      2056    2064  2060   
2096     ssp585      tas  CanESM5   r9i1p1f1      2065    2073  2069   
2097     ssp585      tas  CanESM5   r9i1p1f1      2074    2082  2078   
2098     ssp585      tas  CanESM5   r9i1p1f1      2083    2091  2087   
2099     ssp585      tas  CanESM5   r9i1p1f1      2092    2100  2096   

            fx        dx  
0    -1.360399  0.016472  
75   -1.299509 -0.001848  
76   -1.225040  0.010931  
77   -1.205325 -0.012385  


# Target
It's easiest to just only have a target start in 1858 and go to 2100. For GCAM,
that's fine. Mostly, 1975-2100 is the window you'd need covered for GCAM and
that segments into 9 year chunks perfectly.

In [ ]:
targ = pd.read_csv(os.path.join(data_directory, "tas-data", "CanESM5_tas.csv"))
target_data = targ.loc[(targ["model"] == "CanESM5")
                       & (targ["experiment"] == 'ssp245')].copy()

target_data  = target_data[target_data["ensemble"].isin(['r1i1p1f1'])].copy()

target_data = target_data.drop(columns='zstore').reset_index(drop=True)


target_data = stitches.fx_processing.calculate_rolling_mean(target_data,
                                                            size=31).copy()

The default chunking, starting with 1850 and cutting every 9 years after that.
This results in the final window only being 8 years. That is fine, a target window
that is only 1 year short is not a problem (shorter than that is also not a problem
for the method, we just haven't coded that in).

In [42]:
target_data1 = stitches.fx_processing.get_chunk_info(
    stitches.fx_processing.chunk_ts(df = target_data,  n=9)).copy()
print(target_data1)

   experiment variable  ensemble    model  start_yr  end_yr  year        fx  \
0      ssp245      tas  r1i1p1f1  CanESM5      1850    1858  1854 -1.302066   
1      ssp245      tas  r1i1p1f1  CanESM5      1859    1867  1863 -1.291443   
2      ssp245      tas  r1i1p1f1  CanESM5      1868    1876  1872 -1.269637   
3      ssp245      tas  r1i1p1f1  CanESM5      1877    1885  1881 -1.111034   
4      ssp245      tas  r1i1p1f1  CanESM5      1886    1894  1890 -1.199870   
5      ssp245      tas  r1i1p1f1  CanESM5      1895    1903  1899 -1.106199   
6      ssp245      tas  r1i1p1f1  CanESM5      1904    1912  1908 -1.169673   
7      ssp245      tas  r1i1p1f1  CanESM5      1913    1921  1917 -1.062645   
8      ssp245      tas  r1i1p1f1  CanESM5      1922    1930  1926 -1.072645   
9      ssp245      tas  r1i1p1f1  CanESM5      1931    1939  1935 -1.092245   
10     ssp245      tas  r1i1p1f1  CanESM5      1940    1948  1944 -0.945868   
11     ssp245      tas  r1i1p1f1  CanESM5      1949 

If you'd rather the target window ending in 2100 be a complete 9 years and work
back, you can use the `base_chunk=8` argument to do that. The previous call starting
in 1850 and cutting every 9 years aftere that uses the default `base_chunk=0`.
`base_chunk=8` means the target starts in 1850+8 = 1858 and cuts every 9
years after that, ending in 2100.

If the ESM data ends in 2099 and you wanted the target window ending in 2099
to be a complete 9 year window, you'd use `base_chunk=7`. It's a little clunky and
we could certainly make it more interpretable/flexible, but it's not too bad to
figure out for now.

In [43]:
target_data2 = stitches.fx_processing.get_chunk_info(
    stitches.fx_processing.chunk_ts(df = target_data,  n=9,
                                    base_chunk=8)).copy()
print(target_data2)

   experiment variable  ensemble    model  start_yr  end_yr  year        fx  \
0      ssp245      tas  r1i1p1f1  CanESM5      1858    1866  1862 -1.431094   
1      ssp245      tas  r1i1p1f1  CanESM5      1867    1875  1871 -1.250314   
2      ssp245      tas  r1i1p1f1  CanESM5      1876    1884  1880 -1.096694   
3      ssp245      tas  r1i1p1f1  CanESM5      1885    1893  1889 -1.120996   
4      ssp245      tas  r1i1p1f1  CanESM5      1894    1902  1898 -1.169627   
5      ssp245      tas  r1i1p1f1  CanESM5      1903    1911  1907 -1.206883   
6      ssp245      tas  r1i1p1f1  CanESM5      1912    1920  1916 -1.179884   
7      ssp245      tas  r1i1p1f1  CanESM5      1921    1929  1925 -0.879245   
8      ssp245      tas  r1i1p1f1  CanESM5      1930    1938  1934 -1.061302   
9      ssp245      tas  r1i1p1f1  CanESM5      1939    1947  1943 -0.996936   
10     ssp245      tas  r1i1p1f1  CanESM5      1948    1956  1952 -0.894393   
11     ssp245      tas  r1i1p1f1  CanESM5      1957 

if you really don't want to sacrifice the 1850-1857 years, you can get
kind of clunky and play around with the `chunk_ts` arguments to get an
1850-1857 chunk and just append that to the target data

In [51]:
short_target_window = stitches.fx_processing.get_chunk_info(
    stitches.fx_processing.chunk_ts(df = target_data,  n=8,
                                    base_chunk=0)).copy()

short_target_window = short_target_window[short_target_window['end_yr'] == 1857].copy()

print(short_target_window)

  experiment variable  ensemble    model  start_yr  end_yr  year        fx  \
0     ssp245      tas  r1i1p1f1  CanESM5      1850    1857  1854 -1.302066   

         dx  
0 -0.012558  


In [53]:
target_data3 = pd.concat([short_target_window, target_data2])
print(target_data3)


   experiment variable  ensemble    model  start_yr  end_yr  year        fx  \
0      ssp245      tas  r1i1p1f1  CanESM5      1850    1857  1854 -1.302066   
0      ssp245      tas  r1i1p1f1  CanESM5      1858    1866  1862 -1.431094   
1      ssp245      tas  r1i1p1f1  CanESM5      1867    1875  1871 -1.250314   
2      ssp245      tas  r1i1p1f1  CanESM5      1876    1884  1880 -1.096694   
3      ssp245      tas  r1i1p1f1  CanESM5      1885    1893  1889 -1.120996   
4      ssp245      tas  r1i1p1f1  CanESM5      1894    1902  1898 -1.169627   
5      ssp245      tas  r1i1p1f1  CanESM5      1903    1911  1907 -1.206883   
6      ssp245      tas  r1i1p1f1  CanESM5      1912    1920  1916 -1.179884   
7      ssp245      tas  r1i1p1f1  CanESM5      1921    1929  1925 -0.879245   
8      ssp245      tas  r1i1p1f1  CanESM5      1930    1938  1934 -1.061302   
9      ssp245      tas  r1i1p1f1  CanESM5      1939    1947  1943 -0.996936   
10     ssp245      tas  r1i1p1f1  CanESM5      1948 